**Packages**

In [1]:
! pip install datasets --quiet
! pip install evaluate --quiet
! pip install rouge_score --quiet
! pip install sacrebleu --quiet
! pip install transformers --quiet
! pip install -q sentencepiece --quiet
! pip install summarizer --quiet
! pip install bert-extractive-summarizer --quiet

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import math

from datasets import load_dataset
import evaluate

import inspect

#let's make longer output readable without horizontal scrolling
from pprint import pprint

import warnings

import regex as re

import os, re
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# These auto classes load the right type of tokenizer and model based on a model name
from transformers import AutoTokenizer, TFAutoModel
from transformers import pipeline
from transformers import AutoModel

**Necessary Functions**

In [3]:
rouge = evaluate.load('rouge')

In [4]:
chrf = evaluate.load("chrf")

In [5]:
def get_default_args(func):
    signature = inspect.signature(func)
    return {
        k: v.default
        for k, v in signature.parameters.items()
        if v.default is not inspect.Parameter.empty
    }

**Huggingface Transformers Training Resources**

https://github.com/huggingface/transformers/blob/main/examples/pytorch/summarization/run_summarization.py

https://www.databricks.com/blog/2023/03/20/fine-tuning-large-language-models-hugging-face-and-deepspeed.html

https://gitlab.com/nicolalandro/summarization

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [7]:
df = pd.read_csv('/content/drive/MyDrive/W266FinalProject/Datasets/xl_sum_sample_test.csv')
# df.head(5)

In [8]:
df_tech = pd.read_csv('/content/drive/MyDrive/W266FinalProject/Datasets/xl_sum_sample_test_technology.csv')

In [16]:
def generate_scores(mod, data, do_sample = False, num_beams = 1, top_k = 50, num_beam_groups = 1):

  targets = []
  candidates = []
  r1 = []
  r2 = []
  rL = []
  rLs = []
  chrfs = []

  for i in range(int(len(data['text']))):

      candidate = mod(data['text'][i], 
                              truncation = True,
                              max_length = 256,
                              min_length = 0, 
                              do_sample = do_sample,
                              num_beams = num_beams, 
                              top_k = top_k,
                              num_beam_groups = num_beam_groups,
                              )[0]
      candidate = [candidate['summary_text']]

      ref = [data['summary'][i]]

      results = rouge.compute(predictions=candidate,
                              references=ref)

      targets.append(ref)
      candidates.append(candidate)
      r1.append(results['rouge1'])
      r2.append(results['rouge2'])
      rL.append(results['rougeL'])
      rLs.append(results['rougeLsum'])

      results = chrf.compute(predictions=candidate,
                              references=ref)

      chrfs.append(results['score'])
  
  return pd.DataFrame({'target': targets, 'candidate': candidates,
                  'rouge1': r1, 'rouge2': r2, 'rougeL': rL, 
                      'rougeLs': rLs, 'chrf': chrfs})

# Baseline

In [25]:
def baseline_scores(data):
  base_r1 = []
  base_r2 = []
  base_rL = []
  base_rLs = []
  base_chrf = []
  candidates = []
  targets = []
  for i in range(len(data['text'])): 
      
      
      # first three sentences 
      candidate = ". ".join(data["text"][i].split('. ')[0:3]) + "."
      candidate = [candidate]
      
      ref = [data['summary'][i]]
      
      results = rouge.compute(predictions=candidate,
                              references= ref)
      
      results2 = chrf.compute(predictions=candidate,
                              references= ref)
      
      candidates.append(candidate)
      targets.append(ref)
      base_r1.append(results['rouge1'])
      base_r2.append(results['rouge2'])
      base_rL.append(results['rougeL'])
      base_rLs.append(results['rougeLsum'])
      
      base_chrf.append(results2['score'])
      
  return pd.DataFrame({'target': targets, 'candidate': candidates,
                'rouge1': base_r1, 'rouge2': base_r2, 'rougeL': base_rL, 
                    'rougeLs': base_rLs, 'chrf': base_chrf})

In [38]:
baseline_score_df = baseline_scores(df)

In [39]:
baseline_score_df.to_csv('/content/drive/MyDrive/W266FinalProject/test_candidates/baseline_all_categories.csv')

In [26]:
baseline_tech_score_df = baseline_scores(df_tech)

In [27]:
baseline_tech_score_df.to_csv('/content/drive/MyDrive/W266FinalProject/test_candidates/baseline_tech.csv')

# T5 SCORES

### Base

In [17]:
t5_base_summarizer = pipeline("summarization", model="t5-base")

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
t5_base_scores_df = generate_scores(t5_base_summarizer, df, do_sample = False, num_beams = 4, top_k = 75, num_beam_groups = 2)

In [ ]:
t5_base_scores_df.to_csv('/content/drive/MyDrive/W266FinalProject/test_candidates/t5_all_categories_base.csv')

Technology category

In [20]:
t5_base_tech_scores_df = generate_scores(t5_base_summarizer, df_tech, do_sample = False, num_beams = 4, top_k = 75, num_beam_groups = 2)

Your max_length is set to 256, but you input_length is only 204. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=102)
Your max_length is set to 256, but you input_length is only 192. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=96)
Your max_length is set to 256, but you input_length is only 240. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=120)


In [21]:
t5_base_tech_scores_df.to_csv('/content/drive/MyDrive/W266FinalProject/test_candidates/t5_tech_base.csv')

### Finetuned

In [22]:
t5_finetuned_summarizer = pipeline("summarization", model="arisanguyen/finetuned_T5_all_categories", revision = 'model_0')

In [ ]:
t5_scores_df = generate_scores(t5_finetuned_summarizer, df, do_sample = False, num_beams = 4, top_k = 75, num_beam_groups = 2)

In [ ]:
t5_scores_df.to_csv('/content/drive/MyDrive/W266FinalProject/test_candidates/t5_all_categories_finetuned.csv')

Technology category

In [23]:
t5_tech_scores_df = generate_scores(t5_finetuned_summarizer, df_tech, do_sample = False, num_beams = 4, top_k = 75, num_beam_groups = 2)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Your max_length is set to 256, but you input_length is only 204. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=102)
Your max_length is set to 256, but you input_length is only 192. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=96)
Your max_length is set to 256, but you input_length is only 240. You might consider decreasing max_length manually, e.g. summarizer('...', max_lengt

In [24]:
t5_tech_scores_df.to_csv('/content/drive/MyDrive/W266FinalProject/test_candidates/t5_tech_finetuned.csv')

# BART SCORES

### Base

In [28]:
def generate_bart_scores(model, data, do_sample = False, num_beams = 1, top_k = 50, num_beam_groups = 1):

  targets = []
  candidates = []
  r1 = []
  r2 = []
  rL = []
  rLs = []
  chrfs = []

  for i in range(int(len(data['text']))):
      candidate = model(data['text'][i], 
                               truncation = True, 
                               max_length = 256, 
                               min_length = 0, 
                               do_sample = do_sample,
                               num_beams = num_beams, 
                               top_k = top_k,
                              #  num_beam_groups = num_beam_groups,
                                )[0]
      candidate = [candidate['summary_text']]

      ref = [data['summary'][i]]

      results = rouge.compute(predictions=candidate,
                              references=ref)

      targets.append(ref)
      candidates.append(candidate)
      r1.append(results['rouge1'])
      r2.append(results['rouge2'])
      rL.append(results['rougeL'])
      rLs.append(results['rougeLsum'])

      results = chrf.compute(predictions=candidate,
                              references=ref)

      chrfs.append(results['score'])
  
  return pd.DataFrame({'target': targets, 'candidate': candidates,
                  'rouge1': r1, 'rouge2': r2, 'rougeL': rL, 
                      'rougeLs': rLs, 'chrf': chrfs})

In [29]:
bart_base_summarizer = pipeline("summarization", model="facebook/bart-base")

In [ ]:
bart_base_scores_df = generate_bart_scores(bart_base_summarizer, df, do_sample = True, num_beams = 4, top_k = 75, num_beam_groups = 2)

Technology category

In [30]:
bart_base_tech_scores_df = generate_bart_scores(bart_base_summarizer, df_tech, do_sample = True, num_beams = 4, top_k = 75, num_beam_groups = 2)

Your max_length is set to 256, but you input_length is only 187. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=93)
Your max_length is set to 256, but you input_length is only 175. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=87)
Your max_length is set to 256, but you input_length is only 217. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=108)
Your max_length is set to 256, but you input_length is only 244. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=122)
Your max_length is set to 256, but you input_length is only 234. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=117)


In [31]:
bart_base_tech_scores_df.to_csv('/content/drive/MyDrive/W266FinalProject/test_candidates/bart_tech_base.csv')

### Finetuned

In [32]:
bart_finetuned_summarizer = pipeline("summarization", model="arisanguyen/finetuned-BART-all-categories", revision = 'model_2')

In [ ]:
bart_scores_df = generate_scores(bart_finetuned_summarizer, df, do_sample = True, num_beams = 4, top_k = 75, num_beam_groups = 2)

In [ ]:
bart_scores_df.to_csv('/content/drive/MyDrive/W266FinalProject/test_candidates/bart_all_categories_finetuned.csv')

Technology category

In [33]:
bart_tech_scores_df = generate_scores(bart_finetuned_summarizer, df_tech, do_sample = False, num_beams = 4, top_k = 75, num_beam_groups = 2)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Your max_length is set to 256, but you input_length is only 187. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=93)
Your max_length is set to 256, but you input_length is only 175. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=87)
Your max_length is set to 256, but you input_length is only 217. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=108)
Your max_length is set to 256, but you input_length is only 244. You might consider decreasing max_length manually, e.g. summarizer('..

In [34]:
bart_tech_scores_df.to_csv('/content/drive/MyDrive/W266FinalProject/test_candidates/bart_tech_finetuned.csv')

# BERT SCORES

In [22]:
def generate_bert_scores(model, data, do_sample = False, num_beams = 1, top_k = 50, num_beam_groups = 1):

  targets = []
  candidates = []
  r1 = []
  r2 = []
  rL = []
  rLs = []
  chrfs = []

  for i in range(int(len(data['text']))):

      candidate = model(data['text'][i], 
                      num_sentences = 2, 
                      min_length = 0,
                      max_length = 256,        
                      ratio = None,
                      use_first = False,
                     )
      candidate = [candidate]
      
      ref = [data['summary'][i]]

      results = rouge.compute(predictions=candidate,
                              references=ref)

      targets.append(ref)
      candidates.append(candidate)
      r1.append(results['rouge1'])
      r2.append(results['rouge2'])
      rL.append(results['rougeL'])
      rLs.append(results['rougeLsum'])

      results = chrf.compute(predictions=candidate,
                              references=ref)

      chrfs.append(results['score'])
  
  return pd.DataFrame({'target': targets, 'candidate': candidates,
                  'rouge1': r1, 'rouge2': r2, 'rogueL': rL, 
                      'rogueLs': rLs, 'chrf': chrfs})

In [11]:
from summarizer import Summarizer
bert_model = Summarizer('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
bert_scores_df = generate_bert_scores(bert_model, df, do_sample = False, num_beams = 4, top_k = 75, num_beam_groups = 2)

In [ ]:
bert_scores_df.to_csv('/content/drive/MyDrive/W266FinalProject/test_candidates/bert_all_categories.csv')

Technology category

In [23]:
bert_tech_scores_df = generate_bert_scores(bert_model, df_tech, do_sample = True, num_beams = 4, top_k = 75, num_beam_groups = 2)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: F

In [24]:
bert_tech_scores_df.to_csv('/content/drive/MyDrive/W266FinalProject/test_candidates/bert_tech.csv')